In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score, GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (accuracy_score, 
                            confusion_matrix, 
                            classification_report,
                            f1_score,
                            plot_confusion_matrix,
                            precision_recall_curve,
                            precision_score,
                            recall_score)

In [86]:
pwd

'C:\\Users\\tsuts\\dsi\\07-week\\Mass-Protests\\users'

In [141]:
mass = pd.read_csv('../data/mass_mobile.csv')
mass_no_country = pd.read_csv('./data/mass')

In [142]:
mass.head(2)

,Unnamed: 0,id,country,ccode,region,protestnumber,protesterviolence,location,protesteridentity,sources,...,social_restrictions,start_date,end_date,target,notes_clean,neg,neu,pos,compound,protest_duration
0,0,201990001,Canada,20,North America,1,0.0,national,unspecified,1. great canadian train journeys into history;...,...,0,1990-01-15,1990-01-15,"[0, 0, 0, 0, 1, 0, 0]",canada s railway passenger system was finally ...,0.087,0.913,0.0,-0.8176,1
1,1,201990002,Canada,20,North America,2,0.0,"Montreal, Quebec",unspecified,1. autonomy s cry revived in quebec the new yo...,...,0,1990-06-25,1990-06-25,"[0, 0, 0, 0, 1, 0, 0]",protestors were only identified as young peopl...,0.000,1.000,0.0,0.0000,1


In [143]:
mass.drop(columns=['Unnamed: 0'], inplace=True)

In [144]:
# all columns for easy copy and paste 
sorted(list(mass.columns))

['accomodation',
 'arrests',
 'beatings',
 'ccode',
 'compound',
 'country',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'country_Croatia',
 'country_Cuba',
 'country_Cyprus',
 'country_Czech Republic',
 'country_Czechoslovakia',
 'country_Denmark',
 'country_Djibouti',
 'country_Dominican Republic',
 'country_Ecuador',
 'country_Eg

In [145]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15179 entries, 0 to 15178
Columns: 234 entries, id to protest_duration
dtypes: float64(10), int64(212), object(12)
memory usage: 27.1+ MB


## Macro Functions 

In [146]:
def state_response_predictor(df, features, target, model):
    '''This function runs a model, makes predictions, calls evaluate_model() to 
    evaluate various train and test scores using those predictions, and uses response_prediction_columns() 
    to create prediction and probability columns'''
    X = features 
    y = df[target]
    
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)
    
    # scale data
    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    # fit model 
    model.fit(X_train_sc, y_train)
    
    # predict on train and test set 
    y_preds_train = model.predict(X_train_sc)
    y_preds_test = model.predict(X_test_sc)
    
    # evaluate various train and test set classification metrics
    f1_score_train, precision_train, recall_train = evaluate_model(y_train, y_preds_train)
    f1_score_test, precision_test, recall_test = evaluate_model(y_test, y_preds_test)
    
#     print(f'{target}- F1_score for {model} model, train set: {f1_score_train}')
#     print(f'{target}- F1_score for {model} model, test set: {f1_score_test}')
#     print(f'{target}- Precision for {model} model, train set: {precision_train}')
#     print(f'{target}- Precision for {model} model, test set: {precision_test}')
#     print(f'{target}- Recall for {model} model, train set: {recall_train}')
#     print(f'{target}- Recall for {model} model, teset set: {recall_test}')

    # call response_prediction_columns() to create prediction and probability columns
    predicted_column, probability_column = response_prediction_columns(model, df, features)
    
#     build_confusion_matrix(y_test, y_preds_test)

    return predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test  

In [147]:
def response_prediction_columns(model, df, features):
    '''Creates predicted and probability columns'''
    
    # scale data on entire data set
    ss = StandardScaler()
    X = features
    X_sc = ss.fit_transform(X)
    
    # make predictions
    predicted_column = model.predict(X_sc)
    probability_column = model.predict_proba(X_sc)[:,1]
    
    return predicted_column, probability_column

In [148]:
def evaluate_model(y_true, y_preds):
    '''Building a function that returns F1, precision, and recall scores '''
    
    f1 = f1_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds)
    recall = recall_score(y_true, y_preds)

    return f1, precision, recall

In [149]:
# def build_confusion_matrix(y_true, y_preds, title): # in progress
    '''Build a confusion matrix'''
#     tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
#     disp = plot_confusion_matrix(model, 
#                                  X_test_sc, 
#                                  y_test, 
# #                                  display_labels=
#                                  cmap="Blues")
    
#     disp.ax_.set_title(title)
#     print(title)
#     print(disp.confusion_matrix)
    
#     plt.show
    
#     return

IndentationError: unexpected indent (<ipython-input-149-3c3e8468805b>, line 2)

In [150]:
list(mass.columns)

['id',
 'country',
 'ccode',
 'region',
 'protestnumber',
 'protesterviolence',
 'location',
 'protesteridentity',
 'sources',
 'notes',
 'protester_id_type',
 'partipants_number',
 'protest_size_category',
 'pop_male',
 'pop_female',
 'pop_total',
 'pop_density',
 'prosperity_2020',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'coun

## GridSearch Section

In [154]:
features = mass.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
       'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
       'end_date', 'target', 'arrests', 'accomodation', 'beatings',
       'crowddispersal', 'ignore', 'killings', 'shootings'])
X = features
y = mass['arrests']

logreg = LogisticRegression()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

params = {
    'penalty' : ['l1','l2'], 
    'C' : [.01, 1, 10],
    'solver' : ['liblinear', 'saga', 'lfgbs'],
    'max_iter' : [100, 1000]
}

gs = GridSearchCV(logreg, param_grid=params, cv=10, verbose=2)

gs.fit(X_train_sc, y_train)

gs.best_score_

Fitting 10 folds for each of 36 candidates, totalling 360 fits
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............

C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 433, in _check_solver
    raise ValueError("Logistic Regression supports only solvers in %s, got"
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got lfgbs.

  warnings.warn("Estimator fit failed. The score on this train-test"


[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............

...

[CV] ..... C=10, max_iter=1000, penalty=l2, solver=saga, total=  18.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=saga ....................


C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 433, in _check_solver
    raise ValueError("Logistic Regression supports only solvers in %s, got"
V

[CV] ..... C=10, max_iter=1000, penalty=l2, solver=saga, total=  18.4s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .... C=10, max_iter=1000, penalty=l2, solver=lfgbs, total=   0.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .... C=10, max_iter=1000, penalty=l2, solver=lfgbs, total=   0.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .... C=10, max_iter=1000, penalty=l2, solver=lfgbs, total=   0.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .... C=10, max_iter=1000, penalty=l2, solver=lfgbs, total=   0.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .... C=10, max_iter=1000, penalty=l2, solver=lfgbs, total=   0.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .... C=10, max_iter=1000, penalty=l2, solver=lfgbs, total=   0.0s
[CV] C=10, max_iter=1000, penalty=l2, solver=lfgbs ...................
[CV] .

C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.8602486762801608

In [164]:
targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']  

grids = []


for target in targets:    
    features = mass.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
           'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
           'end_date', 'target', 'arrests', 'accomodation', 'beatings',
           'crowddispersal', 'ignore', 'killings', 'shootings'])
    X = features
    y = mass[target]

    logreg = LogisticRegression()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)

    params = {
        'penalty' : ['l1','l2'], 
        'C' : [.01, 1, 10],
        'solver' : ['liblinear', 'saga', 'lfgbs'],
        'max_iter' : [100, 1000]
    }
    
    gs = GridSearchCV(logreg, param_grid=params, cv=10, verbose=2)
    #print(y)
    grids.append(gs)
        

0        0
1        0
2        0
3        0
4        1
        ..
15174    0
15175    0
15176    0
15177    0
15178    0
Name: arrests, Length: 15179, dtype: int64
0        0
1        0
2        0
3        1
4        1
        ..
15174    0
15175    0
15176    1
15177    0
15178    0
Name: accomodation, Length: 15179, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
15174    0
15175    0
15176    0
15177    0
15178    0
Name: beatings, Length: 15179, dtype: int64
0        0
1        0
2        0
3        0
4        1
        ..
15174    1
15175    1
15176    0
15177    1
15178    0
Name: crowddispersal, Length: 15179, dtype: int64
0        1
1        1
2        1
3        0
4        0
        ..
15174    0
15175    0
15176    0
15177    0
15178    1
Name: ignore, Length: 15179, dtype: int64
0        0
1        0
2        0
3        0
4        0
        ..
15174    0
15175    1
15176    0
15177    0
15178    0
Name: killings, Length: 15179, dtype: int64
0  

In [165]:
grids

[GridSearchCV(cv=10, estimator=LogisticRegression(),
              param_grid={'C': [0.01, 1, 10], 'max_iter': [100, 1000],
                          'penalty': ['l1', 'l2'],
                          'solver': ['liblinear', 'saga', 'lfgbs']},
              verbose=2),
 GridSearchCV(cv=10, estimator=LogisticRegression(),
              param_grid={'C': [0.01, 1, 10], 'max_iter': [100, 1000],
                          'penalty': ['l1', 'l2'],
                          'solver': ['liblinear', 'saga', 'lfgbs']},
              verbose=2),
 GridSearchCV(cv=10, estimator=LogisticRegression(),
              param_grid={'C': [0.01, 1, 10], 'max_iter': [100, 1000],
                          'penalty': ['l1', 'l2'],
                          'solver': ['liblinear', 'saga', 'lfgbs']},
              verbose=2),
 GridSearchCV(cv=10, estimator=LogisticRegression(),
              param_grid={'C': [0.01, 1, 10], 'max_iter': [100, 1000],
                          'penalty': ['l1', 'l2'],
               

In [171]:

results = {
    'best_cv_scores' : [],
    'best_params' : [],
    'train_f1_scores' : [],
    'test_f1_scores' : [],
    'train_precision_scores' : [], 
    'test_precision_scores' : [],
    'train_recall_scores' : [],
    'test_recall_scores' : []
}

targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']  

for target in targets:    
    features = mass.drop(columns=['country', 'ccode', 'region', 'location','protesteridentity', 'sources',
           'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos',
           'end_date', 'target', 'arrests', 'accomodation', 'beatings',
           'crowddispersal', 'ignore', 'killings', 'shootings'])
    X = features
    y = mass[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

    ss = StandardScaler()
    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    # fit grids
    grid.fit(X_train_sc, y_train)

    # append important grid results
    results['best_params'].append(grid.best_params_)
    results['best_cv_scores'].append(grid.best_score_)

    # make predictions 
    train_preds = grid.predict(X_train_sc)
    test_preds = grid.predict(X_test_sc)

    # various classification metrics
    # f1 scores
    results['train_f1_scores'].append(f1_score(y_train, train_preds))
    results['test_f1_scores'].append(f1_score(y_test, test_preds))

    # precision scores 
    results['train_precision_scores'].append(precision_score(y_train, train_preds))
    results['test_precision_scores'].append(precision_score(y_test, test_preds))

    # recall scores
    results['train_recall_scores'].append(recall_score(y_train, train_preds))
    results['test_recall_scores'].append(recall_score(y_test, test_preds))

Fitting 10 folds for each of 36 candidates, totalling 360 fits
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.2s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l1, solver=liblinear, total=   0.1s
[CV] C=0.01, max_iter=100, penalty=l1, solver=liblinear ..............

C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 433, in _check_solver
    raise ValueError("Logistic Regression supports only solvers in %s, got"
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got lfgbs.

  warnings.warn("Estimator fit failed. The score on this train-test"


[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.5s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............
[CV]  C=0.01, max_iter=100, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=100, penalty=l2, solver=liblinear ..............

...

C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 433, in _check_solver
    raise ValueError("Logistic Regression supports only solvers in %s, got"
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got lfgbs.

  warnings.warn("Estimator fit failed. The score on this train-test"


[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear .............
[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.3s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear .............
[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear .............
[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear .............
[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear .............
[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.3s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear .............
[CV]  C=0.01, max_iter=1000, penalty=l2, solver=liblinear, total=   0.4s
[CV] C=0.01, max_iter=1000, penalty=l2, solver=liblinear ......

C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\tsuts\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 433, in _check_solver
    raise ValueError("Logistic Regression supports only solvers in %s, got"
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got lfgbs.

  warnings.warn("Estimator fit failed. The score on this train-test"


[CV] .. C=1, max_iter=100, penalty=l1, solver=liblinear, total=  26.9s
[CV] C=1, max_iter=100, penalty=l1, solver=liblinear .................
[CV] .. C=1, max_iter=100, penalty=l1, solver=liblinear, total=  30.1s
[CV] C=1, max_iter=100, penalty=l1, solver=liblinear .................


KeyboardInterrupt: 

In [172]:
results_df = pd.DataFrame(results)
results_df

,best_cv_scores,best_params,train_f1_scores,test_f1_scores,train_precision_scores,test_precision_scores,train_recall_scores,test_recall_scores
0,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
1,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
2,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
3,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
4,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
5,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
6,0.860249,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.216421,0.246032,0.593750,0.738095,0.132327,0.147619
7,0.900024,"{'C': 10, 'max_iter': 100, 'penalty': 'l1', 's...",0.097892,0.069841,0.637255,0.550000,0.053018,0.037288
8,0.900024,"{'C': 10, 'max_iter': 100, 'penalty': 'l1', 's...",0.097892,0.069841,0.637255,0.550000,0.053018,0.037288
9,0.900024,"{'C': 10, 'max_iter': 100, 'penalty': 'l1', 's...",0.097892,0.069841,0.637255,0.550000,0.053018,0.037288


## Running a Logistic Regression Model Using Macro Functions
- Implementing three above functions for each state response. Will be tweaking the model throughout.

In [108]:
# define features 

# instantiate model
logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# lr_arrests_1 = state_response_predictor(mass, features_1, 'arrests', lr_1)

In [109]:
# lr_arrests_1, f1_score_train_arrests, f1_score_test_arrests, precision_train, precision_test, recall_train, recall_test = state_response_predictor(mass, 
#                                                                                                                                    features_1, 
#                                                                                                                                    'arrests', 
#                                                                                                                                    LogisticRegression_1)

In [110]:
# mass['arrests_predicted'], mass['arrests_probability'] = response_prediction_columns(lr_arrests_1, mass, features_1)

Now we want to run a LogisticRegression on each of our target variables using the state_response_predictor() function

- Responses: `'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'`

In [111]:
mass['arrested_predicted'], mass['arrested_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'accomodation', logreg)

accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.11160714285714285
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.07523510971786833
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.635593220338983
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5
accomodation- Recall for LogisticRegression(max_iter=1000) model, train set: 0.061174551386623165
accomodation- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.04067796610169491


In [112]:
mass['accomodation_predicted'], mass['accomodation_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'accomodation', logreg)

accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.11160714285714285
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.07523510971786833
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.635593220338983
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5
accomodation- Recall for LogisticRegression(max_iter=1000) model, train set: 0.061174551386623165
accomodation- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.04067796610169491


In [113]:
mass['beatings_predicted'], mass['beatings_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'beatings', logreg)

beatings- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.24060150375939848
beatings- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.186046511627907
beatings- Precision for LogisticRegression(max_iter=1000) model, train set: 0.6808510638297872
beatings- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5517241379310345
beatings- Recall for LogisticRegression(max_iter=1000) model, train set: 0.1461187214611872
beatings- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.11188811188811189


In [114]:
mass['crowddispersal_predicted'], mass['crowddispersal_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'crowddispersal', logreg)

crowddispersal- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.6569570135746606
crowddispersal- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.6399535423925667
crowddispersal- Precision for LogisticRegression(max_iter=1000) model, train set: 0.7136712749615975
crowddispersal- Precision for LogisticRegression(max_iter=1000) model, test set: 0.70550576184379
crowddispersal- Recall for LogisticRegression(max_iter=1000) model, train set: 0.6085931359706576
crowddispersal- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.5855472901168969


In [115]:
mass['ignore_predicted'], mass['ignore_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'ignore', logreg)

ignore- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.7946295375435107
ignore- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.7843907916900618
ignore- Precision for LogisticRegression(max_iter=1000) model, train set: 0.7469284188034188
ignore- Precision for LogisticRegression(max_iter=1000) model, test set: 0.7344900105152471
ignore- Recall for LogisticRegression(max_iter=1000) model, train set: 0.8488389740476552
ignore- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.841566265060241


In [116]:
mass['killings_predicted'], mass['killings_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'killings', logreg)

killings- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.2876712328767123
killings- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.16999999999999998
killings- Precision for LogisticRegression(max_iter=1000) model, train set: 0.6116504854368932
killings- Precision for LogisticRegression(max_iter=1000) model, test set: 0.3695652173913043
killings- Recall for LogisticRegression(max_iter=1000) model, train set: 0.1880597014925373
killings- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.11038961038961038


In [117]:
mass['shootings_predicted'], mass['shootings_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features, 'shootings', logreg)

shootings- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.26806282722513086
shootings- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.23728813559322035
shootings- Precision for LogisticRegression(max_iter=1000) model, train set: 0.6183574879227053
shootings- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5283018867924528
shootings- Recall for LogisticRegression(max_iter=1000) model, train set: 0.1711229946524064
shootings- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.15300546448087432


In [120]:
# Now our mass df has our predictions and probabilities for each row 
mass.head()

,Unnamed: 0,id,country,ccode,region,protestnumber,protesterviolence,location,protesteridentity,sources,...,beatings_predicted,beatings_probability,crowddispersal_predicted,crowddispersal_probability,ignore_predicted,ignore_probability,killings_predicted,killings_probability,shootings_predicted,shootings_probability
0,0,201990001,Canada,20,North America,1,0.0,national,unspecified,1. great canadian train journeys into history;...,...,0,0.000186,0,0.065208,1,0.716539,0,0.003705,0,0.015891
1,1,201990002,Canada,20,North America,2,0.0,"Montreal, Quebec",unspecified,1. autonomy s cry revived in quebec the new yo...,...,0,0.000190,0,0.046310,1,0.754182,0,0.001890,0,0.010443
2,2,201990003,Canada,20,North America,3,0.0,"Montreal, Quebec",separatist parti quebecois,1. quebec protest after queen calls for unity ...,...,0,0.000106,0,0.026878,1,0.796236,0,0.000774,0,0.004157
3,3,201990004,Canada,20,North America,4,1.0,"Montreal, Quebec",mohawk indians,1. indians gather as siege intensifies; armed ...,...,0,0.000517,1,0.509338,0,0.081650,0,0.118788,0,0.126993
4,4,201990005,Canada,20,North America,5,1.0,"Montreal, Quebec",local residents,1. dozens hurt in mohawk blockade protest the ...,...,0,0.000796,0,0.451476,0,0.162104,0,0.026036,0,0.092881


## Create DataFrame of various classification metrics for our model

In [122]:
# create dict
model_performance_dict = {
    'target':[],
    'f1_score_train':[],
    'f1_score_test':[],
    'precision_train':[],
    'precision_test':[],
    'recall_train':[],
    'recall_test':[] 
}

# call state_response_predictor() on each target

targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']    
for target in targets:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features, 
                                                                                                                                target, 
# append dictionary with scores                                                                                                                               logreg)
    model_performance_dict['target'].append(target)
#     model_performance_dict['class_balance'].append(mass[classifier].value_counts(normalize=True))
#     model_performance_dict['model'].append(model)
    model_performance_dict['f1_score_train'].append(f1_score_train)
    model_performance_dict['f1_score_test'].append(f1_score_test)
    model_performance_dict['precision_train'].append(precision_train)
    model_performance_dict['precision_test'].append(precision_test)
    model_performance_dict['recall_train'].append(recall_train)
    model_performance_dict['recall_test'].append(recall_test)
    
    
model_performance_df = pd.DataFrame(model_performance_dict)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.26868408986703346
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.25475285171102663
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.6397379912663755
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.6320754716981132
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.17005223447475334
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.1595238095238095
accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.11160714285714285
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.07523510971786833
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.635593220338983
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5
accomodation- Recall for LogisticRegression(max_iter=1000) model,

In [124]:
# dataframe of classification metrics 
model_performance_df

,target,f1_score_train,f1_score_test,precision_train,precision_test,recall_train,recall_test
0,arrests,0.268684,0.254753,0.639738,0.632075,0.170052,0.159524
1,accomodation,0.111607,0.075235,0.635593,0.500000,0.061175,0.040678
2,beatings,0.240602,0.186047,0.680851,0.551724,0.146119,0.111888
3,crowddispersal,0.656957,0.639954,0.713671,0.705506,0.608593,0.585547
4,ignore,0.794630,0.784391,0.746928,0.734490,0.848839,0.841566
5,killings,0.287671,0.170000,0.611650,0.369565,0.188060,0.110390
6,shootings,0.268063,0.237288,0.618357,0.528302,0.171123,0.153005


In [126]:
predictions_df = mass[['id','arrests', 'arrests_predicted', 'arrests_probability', 'accomodation', 'accomodation_predicted', 'accomodation_probability', 
      'beatings', 'beatings_predicted', 'beatings_probability', 'crowddispersal', 'crowddispersal_predicted', 'crowddispersal_probability', 
      'ignore', 'ignore_predicted', 'ignore_probability', 'killings', 'killings_predicted', 'killings_probability',
      'killings', 'killings_predicted', 'killings_probability']].copy()

In [127]:
predictions_df.head()

,id,arrests,arrests_predicted,arrests_probability,accomodation,accomodation_predicted,accomodation_probability,beatings,beatings_predicted,beatings_probability,...,crowddispersal_probability,ignore,ignore_predicted,ignore_probability,killings,killings_predicted,killings_probability,killings,killings_predicted,killings_probability
0,201990001,0,0,0.104568,0,0,0.174497,0,0,0.000192,...,0.078412,1,1,0.755349,0,0,0.006260,0,0,0.006260
1,201990002,0,0,0.191000,0,0,0.117303,0,0,0.000307,...,0.104708,1,1,0.692979,0,0,0.007246,0,0,0.007246
2,201990003,0,0,0.205189,0,0,0.052175,0,0,0.000267,...,0.096429,1,1,0.687309,0,0,0.008844,0,0,0.008844
3,201990004,0,1,0.533197,1,0,0.156799,0,0,0.000359,...,0.379723,0,0,0.055110,0,0,0.103104,0,0,0.103104
4,201990005,1,0,0.421843,1,0,0.085363,0,0,0.000595,...,0.452339,0,0,0.138899,0,0,0.041307,0,0,0.041307


In [128]:
predictions_df.to_csv('../data/04_predictions_only.csv', index=False)

In [139]:
model_performance_dict = {
    'target':[],
#     'model':[],
    'f1_score_train':[],
    'f1_score_test':[],
    'precision_train':[],
    'precision_test':[],
    'recall_train':[],
    'recall_test':[] 
}

# ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']
# ['arrests_BC', 'accomodation_BC', 'beatings_BC', 'crowddispersal_BC', 'ignore_BC', 'killings_BC', 'shootings_BC']
 
targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']    
    
for target in targets:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features_1, 
                                                                                                                                target, 
                                                                                                                                LogisticRegression_1)
    
    model_performance_dict['target'].append(target)
#     model_performance_dict['class_balance'].append(mass[classifier].value_counts(normalize=True))
#     model_performance_dict['model'].append(model)
    model_performance_dict['f1_score_train'].append(f1_score_train)
    model_performance_dict['f1_score_test'].append(f1_score_test)
    model_performance_dict['precision_train'].append(precision_train)
    model_performance_dict['precision_test'].append(precision_test)
    model_performance_dict['recall_train'].append(recall_train)
    model_performance_dict['recall_test'].append(recall_test)
    
print(model_performance_dict)
    
# model_performance_df = pd.DataFrame(model_performance_dict)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.1408450704225352
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.1310211946050096
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5895196506550219
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5666666666666667
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.07997630331753554
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.07407407407407407
LogisticRegression(max_iter=1000)
accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.03843074459567654
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.053571428571428575
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5454545454545454
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.6
accomodation- Recall for Logis

In [140]:
model_performance_df = pd.DataFrame(model_performance_dict)
model_performance_df

,target,f1_score_train,f1_score_test,precision_train,precision_test,recall_train,recall_test
0,arrests,0.140845,0.131021,0.589520,0.566667,0.079976,0.074074
1,accomodation,0.038431,0.053571,0.545455,0.600000,0.019917,0.028037
2,beatings,0.118812,0.144444,0.656250,0.650000,0.065319,0.081250
3,crowddispersal,0.657023,0.608595,0.737338,0.664975,0.592486,0.561028
4,ignore,0.794378,0.780434,0.733632,0.722594,0.866093,0.848338
5,killings,0.171206,0.124352,0.578947,0.480000,0.100457,0.071429
6,shootings,0.211765,0.172249,0.568966,0.486486,0.130092,0.104651


In [90]:
model_performance_dict = {}
 
for classifier in ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features_1, 
                                                                                                                                classifier, 
                                                                                                                                LogisticRegression_1)
     
    model_performance_dict['Classifier'].append(classifier)
#     model_performance_dict['class_balance'] = mass[classifier].value_counts(normalize=True))
    model_performance_dict['model'] = model
    model_performance_dict['f1_score_train'].append(f1_score_train)
    model_performance_dict['f1_score_test'].append(f1_score_test)
    
    model_performance_dict['precision_train'] = precision_train
    model_performance_dict['precision_test'] = precision_test
    model_performance_dict['recall_train'] = recall_train
    model_performance_dict['recall_test'] = recall_test
    
print(model_performance_dict)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.2027871215761653
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.18148820326678766
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5368956743002544
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5434782608695652
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.125
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10893246187363835
LogisticRegression(max_iter=1000)
accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.03522818254603683
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.041666666666666664
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.4666666666666667
accomodation- Recall for LogisticRegression

In [91]:
model_performance_df = pd.DataFrame(model_performance_dict)
model_performance_df

ValueError: If using all scalar values, you must pass an index

In [75]:
mass['arrests'].value_counts(normalize=True)[1]

0    0.858982
1    0.141018
Name: arrests, dtype: float64

In [76]:
mass['ignore'].value_counts(normalize=True)

1    0.543777
0    0.456223
Name: ignore, dtype: float64

In [77]:
mass['beatings'].value_counts(normalize=True)

0    0.947258
1    0.052742
Name: beatings, dtype: float64

In [78]:
mass['shootings'].value_counts(normalize=True)

0    0.938719
1    0.061281
Name: shootings, dtype: float64

In [79]:
mass['killings'].value_counts(normalize=True)

0    0.945813
1    0.054187
Name: killings, dtype: float64

In [80]:
mass['accomodation'].value_counts(normalize=True)

0    0.89977
1    0.10023
Name: accomodation, dtype: float64

In [81]:
mass['crowddispersal'].value_counts(normalize=True)

0    0.686897
1    0.313103
Name: crowddispersal, dtype: float64

In [129]:
# logreg_cvec_pipe = Pipeline([
#         ('cvec', CountVectorizer(stop_words=all_stop_words)),
#         ('logreg', LogisticRegression())
#     ])

# logreg = LogisticRegression()

# params = {
#     'C': [0.01, 0.1, 1, 10, 100],
#     'penalty' : ['l1', 'l2'],
#     'solver': ['lbfgs', 'saga']
# }

# log_reg_grid = GridSearchCV(logreg, 
#                    params,
#                    cv = 5,
#                    verbose = 2,
#                    n_jobs=2)

# logreg_cvec_grid.fit(X_train, y_train)